In [1]:
# Import basic libraries
from palmerpenguins import load_penguins
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="darkgrid")
import time
from sklearn.preprocessing import LabelEncoder
import os
os.chdir("/Users/hoangthuyduongvu/Desktop/FuzzSIM")


In [2]:
# Import personalized libraries
from fuzz.src.capacity import *
from fuzz.choquet.choquet import *
from fuzz.src.norm import *
from fuzz.src.knn import KNNFuzz
from fuzz.src.sim import S1, S2, S3
from fuzz.optim import *
from fuzz.utils import *
from fuzz.eval import leave_one_out
from fuzz.dataloader import *
from fuzz.choquet.d_choquet import *

# BASIC

In [3]:
# Load data 
iris_data = load_iris()
iris = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)
iris['target'] = iris_data.target
data, labels = iris.iloc[:, :-1].values, iris.iloc[:, -1].values
data = batch_norm(np.array(data, dtype=np.float32))

In [4]:
print(monotonic_check_unit(
    X = Capacity(X=[1,2,3], mu=0.4),
    Y = Capacity(X=[2,3], mu=0.1)
))  # -> True
print(monotonic_check_unit(
    X = Capacity(X=[1,2,3], mu=0.04),
    Y = Capacity(X=[2,3], mu=0.1)
))  # -> False
print(monotonic_check_unit(
    X = Capacity(X=[1,3], mu=0.04),
    Y = Capacity(X=[2,3], mu=0.1)
))  # -> True

True
False
True


In [5]:
mu=generate_capacity(enumerate_permute_unit(data[0]))
for i in range(len(mu)):
    print(f"Capaicity of {mu[i].X} is {mu[i].mu}")
print()
print("Monotonicity check:", monotonic_check(mu))   # -> True

Capaicity of [] is 0
Capaicity of [3] is 0.137139089203676
Capaicity of [2] is 0.1710202591347042
Capaicity of [1] is 0.1726825353849043
Capaicity of [0] is 0.2594524995694957
Capaicity of [3, 2] is 0.30578416930366814
Capaicity of [3, 1] is 0.4275607686625058
Capaicity of [3, 0] is 0.4764465312979016
Capaicity of [2, 1] is 0.535031472575219
Capaicity of [2, 0] is 0.6205086468325617
Capaicity of [1, 0] is 0.6411042753483698
Capaicity of [3, 2, 1] is 0.650726126766266
Capaicity of [3, 2, 0] is 0.7010349450820148
Capaicity of [3, 1, 0] is 0.9215490025457924
Capaicity of [2, 1, 0] is 0.9978869726182438
Capaicity of [3, 2, 1, 0] is 1

Monotonicity check: True


In [6]:
features = data[0]

mobius = generate_mobius(features, 2)
mobius.pop(0)           # Include ø in mobius to generate capacity from it easier
capacity = mobius_to_capacity(mobius, features)

print("Möbius:")
for i in range(len(mobius)):
    print(f"Mobius of {mobius[i].X} is {mobius[i].mu:.3f}")

print("\nDerived Capacity:")
for c in capacity:
    print(f"Capacity of {c.X} is {c.mu:.3f}")


Möbius:
Mobius of [np.float64(1.0)] is 0.823
Mobius of [np.float64(0.6734693646430969)] is 0.365
Mobius of [np.float64(0.24489794671535492)] is 0.643
Mobius of [np.float64(0.0)] is 0.677
Mobius of [np.float64(0.6734693646430969), np.float64(1.0)] is 0.301
Mobius of [np.float64(0.24489794671535492), np.float64(1.0)] is 0.881
Mobius of [np.float64(0.0), np.float64(1.0)] is 0.119
Mobius of [np.float64(0.6734693646430969), np.float64(0.24489794671535492)] is 0.065
Mobius of [np.float64(0.6734693646430969), np.float64(0.0)] is 0.225
Mobius of [np.float64(0.24489794671535492), np.float64(0.0)] is 0.064

Derived Capacity:
Capacity of [] is 0.000
Capacity of [np.float64(1.0)] is 0.198
Capacity of [np.float64(0.6734693646430969)] is 0.088
Capacity of [np.float64(0.24489794671535492)] is 0.154
Capacity of [np.float64(0.0)] is 0.163
Capacity of [np.float64(0.6734693646430969), np.float64(1.0)] is 0.358
Capacity of [np.float64(0.24489794671535492), np.float64(1.0)] is 0.564
Capacity of [np.float64

In [ ]:
def check_fit_mobius(mobius, len_features):
    """
    Notice: We need to pop the empty set (ø) from the mobius list to check the fit of the mobius before using this function.
    """
    return True if len(mobius) == len_features + len_features * (len_features - 1) / 2 else False

In [8]:
len(mobius)

10

In [9]:
check_fit_mobius(mobius, len(data[0]))  # -> True

True